In [1]:
#Functions for parsing columns of the spreadsheet on Santa Clara County School Meals
import re
abbrev_to_index = {
    'm': 0, 'mon': 0, 'monday': 0, 'mondays': 0,
    't': 1, 'tues': 1, 'tuesday': 1, 'tuesdays': 1,
    'w': 2, 'wed': 2, 'wednesday': 2, 'wednesdays': 2,
    'th': 3, 'thr': 3, 'thurs': 3, 'thursday': 3, 'thursdays': 3,
    'f': 4, 'fri': 4, 'friday': 4, 'fridays': 4,
    's': 5, 'sat': 5, 'saturday': 5, 'saturdays': 5,
    'su': 6, 'sun': 6, 'sunday': 6, 'sundays': 6
}

daysOfWeek = {
    '0': 'Monday',
    '1': 'Tuesday',
    '2': 'Wednesday',
    '3': 'Thursday',
    '4': 'Friday',
    '5': 'Saturday',
    '6': 'Sunday',

}
 
def fcnparse_siteaddress(csvaddress):
    myoutput = nominatim.query(csvaddress+ ' California')
    if len(myoutput.toJSON())>0:
        myjson = myoutput.toJSON()[0]
    
        #print(myjson)
        #print(type(myjson))
        full_address = myjson['display_name']
        #print(full_address)
        #print(type(full_address))

        tmp = full_address.split(', ')
        #print(tmp)
       
        if tmp[0].isnumeric():
            address = tmp[0]+" "+tmp[1]
            city = tmp[-5:-4][0]
            state = tmp[-3:-2][0]
            zipcode = tmp[-2:-1][0]            
        elif tmp[1].isnumeric():
            address = tmp[1]+" "+tmp[2]
            city = tmp[-5:-4][0]
            state = tmp[-3:-2][0]
            zipcode = tmp[-2:-1][0]
        else:
            address = csvaddress+" ######CHECK ME######"
            city = "######CHECK ME######"
            zipcode = "######CHECK ME######"
            state = "California"
    else:
        full_address = ""
        address = csvaddress+"######CHECK ME######"
        city = "######CHECK ME######"
        zipcode = "######CHECK ME######"
        state = "California"
        
    return full_address, address, city, state, zipcode

def fcnparse_opendays_opentime(attributesOPENDAYS):
    def parse_days_hours(dh_str):
    dh_str = str(dh_str)
    try:
        days = dh_str.split("@")[0].split("\n")[0].lower()
        found = re.search('(through|-|to)', days)
        if not found:
            day = []
            # search for each day in the days
            for d in abbrev_to_index.keys():
                if len(d) > 1:
                    found = re.search(d, days)
                    if found:
                        day.append(d) 

            days = ",".join(day)
        else:
            days = days.replace("through", "-").replace("'s", "").replace("and", ",").replace(' ', '')
        timing = dh_str.split("@")[1]

        dh_str = days + "," + timing
    except:
        dh_str = ""

    day_cols = [''] * 7
    open_day_time = [''] * 3
    if not isinstance(dh_str, str):
        return day_cols

    dh_str = dh_str.strip()

    if 'call' in dh_str:
        return ['call number'] * 7

    openTime = ""
    for dh in dh_str.split(';'):
        dh = dh.strip()
        dh = re.sub('\.', '', dh)  # Remove periods

        # Parse hours
        hours = re.search('\\d.*[pam]*', dh)
        if not hours:
            return ('','', '')
        else:

            hours_str = dh[hours.start():hours.end()].replace("to", "-").replace("â€“", "-")  # Extract hours
            dh = dh[:hours.start()] + dh[hours.end():]  # And remove from dh

        # Assign hours to the correct day(s)
        days = re.split('[,&|]', dh)
        for day in days:
            day = day.strip()
            if len(day) == 0:  # Empty day
                continue
            elif '-' in day:  # Day range (ex. M-F)
                day_range = day.split('-')
                if len(day_range) == 2 and day_range[0] in abbrev_to_index and day_range[1] in abbrev_to_index:
                    start_index = abbrev_to_index[day_range[0]]
                    end_index = abbrev_to_index[day_range[1]]
                    for i in range(start_index, end_index + 1):
                        day_cols[i] = hours_str
            elif day in abbrev_to_index:  # Single day
                index = abbrev_to_index[day]
                day_cols[index] = hours_str

        timing = ""
        for i in range(0, len(day_cols) - 1):
            if day_cols[i]:
                timing = day_cols[i]
                day_cols[i] = daysOfWeek[str(i)]

        if openTime == "":
            # Start Time
            openTime = str(timing.split("-")[0]).strip()

            # End Time
            closeTime = ""
            try:
                closeTime = str(timing.split("-")[1]).strip()
            except:
                pass

    day_cols = list(set(day_cols))
    if "" in day_cols:
        day_cols.remove("")


    return (",".join(day_cols), openTime, closeTime)

def fcnparse_mealtypes(attributesMEALTYPES):
    #print(attributesMEALTYPES)
    mealoptions = []
    if re.search('breakfast', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Breakfast", ]
    if re.search('lunch', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Lunch", ]  
    if re.search('dinner', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Dinner", ]   

    #print(mealoptions)
    return mealoptions


IndentationError: expected an indented block (<ipython-input-1-959445733930>, line 64)

In [2]:
#Code to scrape Santa Clara County school meal sites
import requests
import csv
from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()

post_url_services = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/services'
post_url_organizations = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/organizations'
post_url_address = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/address'
post_url_schedule = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/schedule'
post_url_phones = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/phones'

post_headers = {
    'Authorization' : 'Bearer keyFaCMgt8zX71GnJ',
    'Content-Type': 'application/json'
}
csvfilename = open('/Users/Ashwin/Documents/BAC/20200919cfsj/Santa Clara Scrape 10_1 - Food School Meal Sites .csv')
csv_f = csv.reader(csvfilename)

#Skip header row
count = 1
for row in csv_f:
    if count == 1:
        break

#Loop over all rows and parse out the columns in each row
count = 1
for row in csv_f:
    #print(row)
    rowcount = 0
    for elem in row:
        print(rowcount, elem)
        rowcount = rowcount+1
    
    #Parse data for PHONES sheet: attributes.PHONENUMBER
    attributesPHONENUMBER = row[14]
    airtablePHONES = {"fields": {"number": attributesPHONENUMBER}}
    
    #Parse data for ORGANIZATIONS sheet: attributes.SITESCHOOLDISTRICT
    attributesSITESCHOOLDISTRICT = row[4]
    airtableORGANIZATIONS = {"fields": {"name": attributesSITESCHOOLDISTRICT}}

    #Parse data for SERVICES sheet: attributes.SITENAME, attributes.MEALTYPES, 
    #attributes.SITESTATUS, attributes.DESCRIPTION, attributes.EMAIL, attributes.WEBLINK
    attributesSITENAME = row[5]
    attributesMEALTYPES = fcnparse_mealtypes(row[10])
    attributesSITESTATUS = row[12]
    attributesDESCRIPTION = row[13]+" "+row[23]    
    attributesEMAIL = row[15]
    attributesWEBLINK = row[16]
    airtableSERVICES = {"fields": {"Name": attributesSITENAME, 
                                   "freefood_area": attributesMEALTYPES, 
                                   "active": attributesSITESTATUS,
                                   "Description": attributesDESCRIPTION,
                                   "email": attributesEMAIL,
                                   "url": attributesWEBLINK}}

    
    #Parse data for ADDRESS sheet: attributes.SITEADDRESS   
    attributesSITEADDRESS = row[6]
    #print(attributesSITEADDRESS)
    full_address, address, city, state, zipcode = fcnparse_siteaddress(attributesSITEADDRESS)
    #print(full_address)
    #print(count+1, address, city, state, zipcode, '\n')
    airtableADDRESS = {"fields": {"address_1": address,"city": city,"State": state,"Zip Code": zipcode}} 
        
    #Parse data for SCHEDULE sheet: attributes.OPENDAYS, attributes.OPENTIMES
    attributesOPENDAYS = row[7]
    attributesOPENTIMES = row[8]
    opendaystime = fcnparse_opendays_opentime(attributesOPENDAYS+ '@'+ attributesOPENTIMES)
    airtableSCHEDULE = {"fields": {"weekday": opendaystime(0), "opens_at": opendaystime(1), "closes_at": opendaystime(2)}} 
    
    #Upload to airtable
    print((requests.post(post_url_phones, headers = post_headers, json = airtablePHONES)).status_code)    
    ((requests.post(post_url_organizations, headers = post_headers, json = airtableORGANIZATIONS)).status_code)    
    print((requests.post(post_url_services, headers = post_headers, json = airtableSERVICES)).status_code)    
    ((requests.post(post_url_address, headers = post_headers, json = airtableADDRESS)).status_code)    
    ((requests.post(post_url_schedule, headers = post_headers, json = airtableSCHEDULE)).status_code)

    count = count + 1
    if count == 2:
        break    

ModuleNotFoundError: No module named 'OSMPythonTools'

In [27]:
#Prototyping code for parsing addresses

from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()
#myoutput = nominatim.query('1680 Foley ave')
#print(myoutput.toJSON())
csvaddress = '592 Dunholme Way, san jose'
myoutput = nominatim.query(csvaddress)
print(myoutput)
print(myoutput.toJSON())
print(len(myoutput.toJSON()))
myjson = myoutput.toJSON()[0]
print(myjson)
print(type(myjson))
full_address = myjson['display_name']
print(full_address)
print(type(full_address))

tmp = full_address.split(', ')
print(tmp)
firstelem = tmp[0]
print(firstelem)
print(firstelem.isnumeric())
if tmp[0].isnumeric():
    address = tmp[0]+" "+tmp[1]
    city = tmp[-5:-4][0]
    state = tmp[-3:-2][0]
    zipcode = tmp[-2:-1][0]            
elif tmp[1].isnumeric():
    address = tmp[1]+" "+tmp[2]
    city = tmp[-5:-4][0]
    state = tmp[-3:-2][0]
    zipcode = tmp[-2:-1][0]
else:
    address = csvaddress+" ######CHECK ME######"
    city = "######CHECK ME######"
    zipcode = "######CHECK ME######"
    state = "California"

print(address)
print(city)
print(state)
print(zipcode)

[nominatim] downloading data: 592 Dunholme Way, san jose
[]
0


IndexError: list index out of range

In [87]:
if ~len([]):
    print("yup empty")

yup empty


In [123]:
#blah = '1680'
#blah.isdigit()
print(1!=2)

True


In [11]:
#Prototyping code for parsing meal types

import re

if re.search('mandye', 'Mandy Pande', re.IGNORECASE):
    print("Yes found")
    
test_tup1 = [1, 3, 5] 
test_tup2 = [4,]
  
# printing original tuples 
print("The original tuple 1 : " + str(test_tup1)) 
print("The original tuple 2 : " + str(test_tup2)) 
  
# Ways to concatenate tuples 
# using + operator 
res = test_tup1 + test_tup2 
  
# printing result 
print("The tuple after concatenation is : " + str(res))

The original tuple 1 : [1, 3, 5]
The original tuple 2 : [4]
The tuple after concatenation is : [1, 3, 5, 4]
